# Usando las gráficas... 

<img src="Mexico_City_metro.png"> 

In [44]:
include("lineas_metro.txt")

using LightGraphs
using GraphLayout
;

In [2]:
#Condiciones iniciales
length_inic = Length_Lineas()
est_tot_inic = Length_Estaciones()

195

Creamos la función tipo `Graph` **MetroNetwork**. <br>
Una **red** será representada por  **vértices** y se conectará a través de **artistas** o **edges**. La notación para una gráfica es <br>
`Graph(V,E) = {V::vertices,E::edges}` 

In [3]:
MetroNetwork(total_lines=Todas_Lineas) = Graph(Length_Estaciones(total_lines))
typeof(MetroNetwork())

Graph (constructor with 7 methods)

Conectamos por aristas las estaciones de cada línea de metro de manera *independiente*. Es decir, sin unir los trasbordos.

In [4]:
# Dado el arreglo de líneas de metro, construimos una red
# y conectamos sus vértices [estación a estación]
# sin tomar en cuenta los trasbordos

function ConnectSimpleNetwork(total_lines=Todas_Lineas)
    metro_network = MetroNetwork(total_lines)
    lines_length  = Length_Lineas(total_lines) 
    
    s = 1
    for i = 1:length(lines_length)
        for station in s:lines_length[i] + s - 2
            add_edge!(metro_network, station, station + 1)
            s += 1
        end
        s += 1
    end
    return metro_network
end

ConnectSimpleNetwork (generic function with 2 methods)

In [5]:
#Obs: 195 - 183 = 12 [líneas independientes de la red]
metro_network_simple = ConnectSimpleNetwork()

{195, 183} undirected graph

Usando la función `GetStationNumber`, y `GetStationName`, hacemos una función que encuentre los puntos de `metro_network_simple` cuyos nombres sean iguales y los una con una arista.
Esta arista representa los trasbordos de una estación a otra.

In [6]:
function ConnectLines(total_lines=Todas_Lineas)
    num_est_tot   = Length_Estaciones(total_lines)
    lines_length  = Length_Lineas(total_lines)
    #metro_network = ConnectSimpleNetwork(total_lines)
    metro_network = MetroNetwork(total_lines)
    for i = 1:num_est_tot
        for j = 1:num_est_tot
            if i == j 
                continue 
                elseif GetStationName(i, total_lines) == GetStationName(j, total_lines)
                if has_edge(metro_network, i, j)
                else
                    add_edge!(metro_network, i, j)
                end
            end
        end
    end
    return metro_network
end

ConnectLines (generic function with 2 methods)

In [7]:
#Obs: 220 - 183 = 37 transbordos en toda la red.
metro_network_transfer = ConnectLines()

{195, 37} undirected graph

Falta incluir lo de **Weigthed Network** que no lo entendí 100% bien. 
Sé que hay que darle un distinto "peso" para distinguir el tipo de aristas, pero no entiendo bien cómo...

Ahora, se le dará un **peso** a las aristas, que cumplirá la función de tiempo de espera, distinguiéndolas en dos tipos: <br> 
 * Arista tipo trasbordo, que denominaremos `transit_edge` <br>
 * Arista tipo conexión normal, que denominaremos `normal_edge`. 


In [8]:
function ConnectFullNetwork_Weighted(normal_edge=1, transit_edge=1.1,total_lines=Todas_Lineas)
    num_est_tot   = Length_Estaciones(total_lines)
    lines_length  = Length_Lineas(total_lines)
    metro_network = MetroNetwork(total_lines)
    edge_dists = zeros(num_est_tot,num_est_tot) #Matriz que guarda las distancias entre aristas
    
    #ConnectSimpleNetwork, "pesando" los ejes
    s = 1
    for i = 1:length(lines_length)
        for station in s:lines_length[i] + s - 2
            add_edge!(metro_network, station, station + 1)
            edge_dists[station, station + 1] = normal_edge 
            edge_dists[station + 1, station] = normal_edge
            s += 1
        end
        s += 1
    end
    
    #ConnectLines, pesando los trasbordos 
    for i = 1:num_est_tot
        for j = 1:num_est_tot
            if GetStationName(i, total_lines) == GetStationName(j, total_lines)
                if i != j
                    if has_edge(metro_network, i, j)
                    else
                        add_edge!(metro_network, i, j)
                        edge_dists[i, j] = transit_edge
                        edge_dists[j, i] = transit_edge
                    end
                end
            end
        end
    end
    return metro_network, edge_dists
end

ConnectFullNetwork_Weighted (generic function with 4 methods)

Ahora necesitamos hacer esto más amigable.
Es decir, debemos crear una función que, dada una estación inicial y una final, nos diga la distancia de una a otra.

In [42]:
ConnectFullNetwork_Weighted(1,1.1,Todas_Lineas);

In [14]:
function MatrixDist(transfer_list, total_lines=Todas_Lineas)
    
    MetroNetwork_Weighted, Edge_Dist = ConnectFullNetwork_Weighted(1,1.1,total_lines)
    
    N = Length_Estaciones()
    D = dijkstra_shortest_paths(MetroNetwork_Weighted, 1, edge_dists= Edge_Dist).dists;
    for i = 2:N
        d = dijkstra_shortest_paths(MetroNetwork_Weighted, i, edge_dists= Edge_Dist).dists
        D = hcat(D, d)
    end

    for i = 1:N
        for t1 in transfer_list
            for t2 in transfer_list
                if GetStationName(t1) == GetStationName(t2)
                    
                    if  D[i,t2] < D[i,t1]
                        D[i,t1] = D[i,t2]
                    if  D[t2,i] < D[t1,i]
                        D[t1,i] = D[t2,i]
                    end
                    end
                end
            end
        end
    end
    return D
end

MatrixDist (generic function with 2 methods)

In [41]:
@time D = MatrixDist(Transfer_List);

elapsed time: 1.346749894 seconds (675609872 bytes allocated, 45.20% gc time)


In [47]:
#Sacamos tupla con número de estaciones recorridas y número de trasbordos tomados.
function NormalesyTrasbordos(matrix_dist, est_init, est_final,transit_edge_dist=1.1, normal_edge_dist=1.)
    num_tras = round(matrix_dist[est_init, est_final]%1, 1)*10
    num_est = round((matrix_dist[est_init, est_final] - transit_edge_dist*num_tras)/normal_edge_dist, 1)
    return num_est, num_tras
end

NormalesyTrasbordos (generic function with 3 methods)